In [ ]:
import os
import glob 
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tensorflow import keras
from keras.models import Sequential, Model, load_model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, activation
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from keras import backend as k
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
train_directory = "C:\\Users\\Admin\Desktop\\archive\\Fundus_Train_Val_Data\\Fundus_Scanes_Sorted\\train"
test_directory = "C:\\Users\\Admin\\esktop\\archive\\Fundus_Train_Val_Data\\Fundus_Scanes_Sorted\\test"

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,                   
                                   validation_split=0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(
train_directory, 
target_size=(256,256), 
color_mode='rgb', 
batch_size=64, 
class_mode='categorical', 
subset='training',
shuffle=True,
seed=42
)
validation_generator = train_datagen.flow_from_directory(
train_directory,
target_size=(256,256), 
color_mode='rgb', 
batch_size=64, 
class_mode='categorical', 
subset='validation', 
shuffle=False
)


In [32]:
print(os.listdir("C:\\Users\\Admin\\Desktop\\archive\\Fundus_Train_Val_Data\\Fundus_Scanes_Sorted"))

['test', 'train']


In [38]:
data_dir =Path("C:\\Users\\Admin\\Desktop\\archive\\Fundus_Train_Val_Data\\Fundus_Scanes_Sorted")
train_dir= data_dir/'train'
test_dir=data_dir/'test'



In [ ]:
def load_train():
    normal_cases_dir = train_dir /'Glaucoma_Negative'
    affected_cases_dir = train_dir /'Glaucoma_Positive'
    normal_cases = normal_cases_dir.glob('*.jpg')
    affected_cases = affected_cases_dir.glob('*.jpg')
    train_data=[]
    train_label=[]
    for img in normal_cases:
        train_data.append(img)
        train_label.append('Glaucoma_Negative')
    for img in affected_cases:
        train_data.append(img)
        train_label.append('Glaucoma_Positive')
    df=pd.DataFrame(train_data)
    
    df['labels']=train_label
    df=df.sample(frac=1).reset_index(drop=True)
    return df
    


In [66]:
train_data=load_train()
train_data.shape

(520, 2)

In [ ]:
plt.bar(train_data['labels'].value_counts().index,train_data['labels'].value_counts().values)
plt.show()

In [ ]:
def plot(image_batch, label_batch):
    plot.figure(figsize=(10,5))
    for i in range(10):
        ax = plt.subplot(2,5,i+1)
        img = cv2.imread(str(image_batch[i]))
        img = cv2.resize(img, (224,224))
        plt.imshow(img)
        plt.title(label_batch[i])
        plt.axis("off")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D
#from keras.layers.normalization import BatchNormalization

from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = Sequential()

model.add(Conv2D(64, (9, 9), input_shape=(100,100,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=30,
                                   shear_range=0.2,
                                   zoom_range=[0.8, 1.2],
                                    horizontal_flip=True,
#                                     vertical_flip = True,
                                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory("C:\\Users\\Admin\\Desktop\\archive\\Fundus_Train_Val_Data\\Fundus_Scanes_Sorted\\train",
                                                 target_size = (100, 100),
                                                 batch_size = 64,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory("C:\\Users\\Admin\\Desktop\\archive\\Fundus_Train_Val_Data\\Fundus_Scanes_Sorted\\test",
                                            target_size = (100, 100),
                                            batch_size = 64,
                                            class_mode = 'binary')
my_callbacks = [
    # tf.keras.callbacks.EarlyStopping(patience=4, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('my_model2.h5', 
    verbose=1, save_best_only=True, save_weights_only=False) 
    ]

model.fit(training_set, epochs=100, validation_data = test_set, callbacks=my_callbacks)

model.save('my_model3.h5')

In [ ]:
import streamlit as st
import tensorflow as tf
from PIL import Image, ImageOps
import numpy as np

st.set_option('deprecation.showfileUploaderEncoding', False)

# @st.cache(suppress_st_warning=True,allow_output_mutation=True)
def import_and_predict(image_data, model):
    image = ImageOps.fit(image_data, (100,100),Image.ANTIALIAS)
    image = image.convert('RGB')
    image = np.asarray(image)
    st.image(image, channels='RGB')
    image = (image.astype(np.float32) / 255.0)
    img_reshape = image[np.newaxis,...]
    prediction = model.predict(img_reshape)
    return prediction

model = tf.keras.models.load_model('my_model2.h5')

st.write("""
         # ***Glaucoma detector***
         """
         )

st.write("This is a simple image classification web app to predict glaucoma through fundus image of eye")

file = st.file_uploader("Please upload an image(jpg) file", type=["jpg"])

if file is None:
    st.text("You haven't uploaded a jpg image file")
else:
    imageI = Image.open(file)
    prediction = import_and_predict(imageI, model)
    pred = prediction[0][0]
    if(pred > 0.5):
        st.write("""
                 ## **Prediction:** Your eye is Healthy. Great!!
                 """
                 )
        st.balloons()
    else:
        st.write("""
                 ## **Prediction:** You are affected by Glaucoma. Please consult an ophthalmologist as soon as possible.
                 """
                 )